In [ ]:
## PREPARATIONS BEFORE PARALLEL
const USE_GPU     = false                       # Whether to use GPU
using ParallelStencil
using ParallelStencil.FiniteDifferences3D
@static if USE_GPU
    @init_parallel_stencil(CUDA,    Float64, 3)
else
    @init_parallel_stencil(Threads, Float64, 3)
end
using Images, FileIO, Interpolations, ColorTypes, TestImages, Plots


## CREATE PARALLEL FUNCTIONS TO BE USED IN THE PSEUDO-TRANSIENT LOOP
@views av_xya(A)  = (A[1:end-1, 1:end-1, :] .+ A[2:end, 1:end-1, :] .+ A[1:end-1, 2:end, :] .+ A[2:end, 2:end, :]).*0.25
@views av_xza(A)  = (A[1:end-1, :, 1:end-1] .+ A[1:end-1, :, 2:end] .+ A[2:end, :, 1:end-1] .+ A[2:end, :, 2:end]).*0.25
@views av_yza(A)  = (A[:, 1:end-1, 1:end-1] .+ A[:, 1:end-1, 2:end] .+ A[:, 2:end, 1:end-1] .+ A[:, 2:end, 2:end]).*0.25

@parallel function compute_Vgrad!(dVxdx::Data.Array, dVydy::Data.Array, dVzdz::Data.Array, dVxdy::Data.Array, dVxdz::Data.Array, dVydx::Data.Array, dVydz::Data.Array, dVzdx::Data.Array, dVzdy::Data.Array, Vx::Data.Array, Vy::Data.Array, Vz::Data.Array, dx::Data.Number, dy::Data.Number, dz::Data.Number)
    @all(dVxdx)   = @d_xa(Vx)/dx
    @all(dVydy)   = @d_ya(Vy)/dy
    @all(dVzdz)   = @d_za(Vz)/dz
    @all(dVxdy)   = @d_ya(Vx)/dy
    @all(dVxdz)   = @d_za(Vx)/dz
    @all(dVydx)   = @d_xa(Vy)/dx
    @all(dVydz)   = @d_za(Vy)/dz
    @all(dVzdx)   = @d_xa(Vz)/dx
    @all(dVzdy)   = @d_ya(Vz)/dy
    return
end

@parallel function compute_P!(divV::Data.Array, P::Data.Array, dVxdx::Data.Array, dVydy::Data.Array, dVzdz::Data.Array, Gdτ::Data.Array, r::Data.Number, dx::Data.Number, dy::Data.Number, dz::Data.Number)
    @all(divV)    = @av_yza(dVxdx) + @av_xza(dVydy) + @av_xya(dVzdz)
    @all(P)       = @all(P) - r*@all(Gdτ)*@all(divV)
    return
end

@parallel function compute_E!(Exx::Data.Array, Eyy::Data.Array, Ezz::Data.Array, Exy::Data.Array, Exz::Data.Array, Eyz::Data.Array, dVxdx::Data.Array, dVydy::Data.Array, dVzdz::Data.Array, dVxdy::Data.Array, dVxdz::Data.Array, dVydx::Data.Array, dVydz::Data.Array, dVzdx::Data.Array, dVzdy::Data.Array)
    @all(Exx)     = @av_yza(dVxdx)
    @all(Eyy)     = @av_xza(dVydy)
    @all(Ezz)     = @av_xya(dVzdz)
    @all(Exy)     = 0.5 * ( @av_xza(dVxdy) + @av_yza(dVydx) )
    @all(Exz)     = 0.5 * ( @av_xya(dVxdz) + @av_yza(dVzdx) )
    @all(Eyz)     = 0.5 * ( @av_xya(dVydz) + @av_xza(dVzdy) )
    return
end

@parallel function compute_τ!(sigmaxx::Data.Array, sigmayy::Data.Array, sigmazz::Data.Array, P::Data.Array, τxx::Data.Array, τyy::Data.Array, τzz::Data.Array, τxy::Data.Array, τxz::Data.Array, τyz::Data.Array, τxx_o::Data.Array, τyy_o::Data.Array, τzz_o::Data.Array, τxy_o::Data.Array, τxz_o::Data.Array, τyz_o::Data.Array, Exx::Data.Array, Eyy::Data.Array, Ezz::Data.Array, Exy::Data.Array, Exz::Data.Array, Eyz::Data.Array, etan::Data.Array, Gdτ::Data.Array)
    @all(τxx_o)   = @all(τxx)
    @all(τyy_o)   = @all(τyy)
    @all(τzz_o)   = @all(τzz)
    @all(τxy_o)   = @all(τxy)
    @all(τxz_o)   = @all(τxz)
    @all(τyz_o)   = @all(τyz)
    @all(τxx)     = ( @all(τxx_o) + 2.0*@all(Gdτ)*@all(Exx) )  /  ( @all(Gdτ)/@all(etan) + 1.0 )
    @all(τyy)     = ( @all(τyy_o) + 2.0*@all(Gdτ)*@all(Eyy) )  /  ( @all(Gdτ)/@all(etan) + 1.0 )
    @all(τzz)     = ( @all(τzz_o) + 2.0*@all(Gdτ)*@all(Ezz) )  /  ( @all(Gdτ)/@all(etan) + 1.0 )
    @all(τxy)     = ( @all(τxy_o) + 2.0*@all(Gdτ)*@all(Exy) )  /  ( @all(Gdτ)/@all(etan) + 1.0 )
    @all(τxz)     = ( @all(τxz_o) + 2.0*@all(Gdτ)*@all(Exz) )  /  ( @all(Gdτ)/@all(etan) + 1.0 )
    @all(τyz)     = ( @all(τyz_o) + 2.0*@all(Gdτ)*@all(Eyz) )  /  ( @all(Gdτ)/@all(etan) + 1.0 )
    @all(sigmaxx) = @all(τxx) + @all(P)
    @all(sigmayy) = @all(τyy) + @all(P)
    @all(sigmazz) = @all(τzz) + @all(P)
    return
end

@parallel function compute_Pτgrad!(dPdx::Data.Array, dPdy::Data.Array, dPdz::Data.Array, dτxxdx::Data.Array, dτyydy::Data.Array, dτzzdz::Data.Array, dτxydx::Data.Array, dτxydy::Data.Array, dτxzdx::Data.Array, dτxzdz::Data.Array, dτyzdy::Data.Array, dτyzdz::Data.Array, P::Data.Array, τxx::Data.Array, τyy::Data.Array, τzz::Data.Array, τxy::Data.Array, τxz::Data.Array, τyz::Data.Array, dx::Data.Number, dy::Data.Number, dz::Data.Number)
    @all(dPdx)    = @d_xa(P)  /dx
    @all(dPdy)    = @d_ya(P)  /dy
    @all(dPdz)    = @d_za(P)  /dz
    @all(dτxxdx)  = @d_xa(τxx)/dx
    @all(dτyydy)  = @d_ya(τyy)/dy
    @all(dτzzdz)  = @d_za(τzz)/dz
    @all(dτxydx)  = @d_xa(τxy)/dx
    @all(dτxydy)  = @d_ya(τxy)/dy
    @all(dτxzdx)  = @d_xa(τxz)/dx
    @all(dτxzdz)  = @d_za(τxz)/dz
    @all(dτyzdy)  = @d_ya(τyz)/dy
    @all(dτyzdz)  = @d_za(τyz)/dz
    return
end

@parallel function compute_dV!(Rx::Data.Array, Ry::Data.Array, Rz::Data.Array, dVx::Data.Array, dVy::Data.Array, dVz::Data.Array, dPdx::Data.Array, dPdy::Data.Array, dPdz::Data.Array, dτxxdx::Data.Array, dτyydy::Data.Array, dτzzdz::Data.Array, dτxydx::Data.Array, dτxydy::Data.Array, dτxzdx::Data.Array, dτxzdz::Data.Array, dτyzdy::Data.Array, dτyzdz::Data.Array, dτ_Rho::Data.Array, PI2::Data.Number, rho_Rock::Data.Array, rho_sc::Data.Number)
    @all(Rx)      = - @av_yza(dPdx) + PI2 * @av_xza(dτxydy) + PI2 * @av_xya(dτxzdz) + PI2 * @av_yza(dτxxdx)
    @all(Ry)      = - @av_xza(dPdy) + PI2 * @av_yza(dτxydx) + PI2 * @av_xya(dτyzdz) + PI2 * @av_xza(dτyydy)
    @all(Rz)      = - @av_xya(dPdz) + PI2 * @av_yza(dτxzdx) + PI2 * @av_xza(dτyzdy) + PI2 * @av_xya(dτzzdz) - PI2 * @av(rho_Rock) / rho_sc
    @all(dVx)     =   @av(dτ_Rho)  * @all(Rx)
    @all(dVy)     =   @av(dτ_Rho)  * @all(Ry)
    @all(dVz)     =   @av(dτ_Rho)  * @all(Rz)
    return
end

@parallel function compute_V!(Vx::Data.Array, Vy::Data.Array, Vz::Data.Array, dVx::Data.Array, dVy::Data.Array, dVz::Data.Array)
    @inn(Vx)      = @inn(Vx) + @all(dVx)
    @inn(Vy)      = @inn(Vy) + @all(dVy)
    @inn(Vz)      = @inn(Vz) + @all(dVz)
    return
end

@parallel function boundary_value!(P_x::Data.Array, P_y::Data.Array, P_z::Data.Array, etan_x::Data.Array, etan_y::Data.Array, etan_z::Data.Array, P::Data.Array, etan::Data.Array)
    @all(P_x)     = @av_yza(P)
    @all(P_y)     = @av_xza(P)
    @all(P_z)     = @av_xya(P)
    @all(etan_x)  = @av_yza(etan)
    @all(etan_y)  = @av_xza(etan)
    @all(etan_z)  = @av_xya(etan)
    return
end

@parallel_indices (ix, iy) function TopBottom!(Vz::Data.Array, dz::Data.Number, p_water::Data.Number, p_NVP::Data.Number, P_z::Data.Array, etan_z::Data.Array)
    Vz[ix+1, iy+1, 1]   = Vz[ix+1, iy+1, 2]     - dz * ( - p_water - p_NVP + P_z[ix, iy, 1] ) / etan_z[ix, iy, 1]   / 2
    Vz[ix+1, iy+1, end] = Vz[ix+1, iy+1, end-1] + dz * ( - p_water + P_z[ix, iy, end]       ) / etan_z[ix, iy, end] / 2
    return
end

@parallel_indices (iy, iz) function LeftRight!(Vx::Data.Array, dx::Data.Number, p_NMHP::Data.Number, P_x::Data.Array, etan_x::Data.Array, P_BC::Data.Array)
    Vx[1, iy+1, iz+1]   = Vx[2, iy+1, iz+1]     - dx * ( - p_NMHP * P_BC[1, iy+1, iz+1]   + P_x[1, iy, iz]   ) / etan_x[1, iy, iz]   / 2
    Vx[end, iy+1, iz+1] = Vx[end-1, iy+1, iz+1] + dx * ( - p_NMHP * P_BC[end, iy+1, iz+1] + P_x[end, iy, iz] ) / etan_x[end, iy, iz] / 2 
    return
end

@parallel_indices (ix, iz) function FrontBack!(Vy::Data.Array, dy::Data.Number, p_NMHP::Data.Number, P_y::Data.Array, etan_y::Data.Array, P_BC::Data.Array)
    Vy[ix+1, 1, iz+1]   = Vy[ix+1, 2, iz+1]     - dy * ( - p_NMHP * P_BC[ix+1, 1, iz+1]   + P_y[ix, 1, iz]   ) / etan_y[ix, 1, iz]   / 2
    Vy[ix+1, end, iz+1] = Vy[ix+1, end-1, iz+1] + dy * ( - p_NMHP * P_BC[ix+1, end, iz+1] + P_y[ix, end, iz] ) / etan_y[ix, end, iz] / 2 
    return
end

function smooth3D(A::Array, factor::Number)
    m, n, h = size(A)
    A2 = zeros(m, n, h)
    for i = 2:m-1
        for j = 2:n-1
            for k = 2:h-1
                A2[i,j,k] = A[i,j,k] + 1.0/6.1/factor*( (A[i+1,j,k]-A[i,j,k])-(A[i,j,k]-A[i-1,j,k]) + (A[i,j+1,k]-A[i,j,k])-(A[i,j,k]-A[i,j-1,k]) + (A[i,j,k+1]-A[i,j,k])-(A[i,j,k]-A[i,j,k-1]) )
            end
        end
    end
    # Copy boundary values
    A2[1,:,:] = A[1,:,:]
    A2[end,:,:] = A[end,:,:]
    A2[:,1,:] = A[:,1,:]
    A2[:,end,:] = A[:,end,:]
    A2[:,:,1] = A[:,:,1]
    A2[:,:,end] = A[:,:,end]
    return A2
end

function compute_maxloc3D(matrix::Array)
    rows, cols, deps = size(matrix)
    result = copy(matrix)
    for r = 1:rows
        for c = 1:cols
            for d = 1:deps
                neighbors = [
                    matrix[max(r-1, 1), c, d],  
                    matrix[min(r+1, rows), c, d], 
                    matrix[r, max(c-1, 1), d],  
                    matrix[r, min(c+1, cols), d],
                    matrix[r, c, max(d-1, 1)],
                    matrix[r, c, min(d+1, deps)]
                ]
                result[r, c, d] = maximum(neighbors)
            end
        end
    end
    return result
end


## MAIN FUNCTION
@views function SaltInclusion3D()
    # Physics
    A_Salt      = 1e17                          # Salt viscosity [Pa-s]
    A_Shale     = 1e15                          # Shale viscosity [Pa-s]
    g           = 9.81                          # Gravity [m/s2]
    p_grad_h    = 0.75*22620.6                  # Minimum horizontal stress gradient [Pa/m]
    p_grad_v    = 0.85*22620.6                  # Vertical stress gradient [Pa/m]
    rho_Shale   = p_grad_v/g                    # Bulk Shale density [kg/m3]
    rho_Salt    = 2200                          # Salt density [kg/m3]
    rho_Water   = 1020                          # Water density [kg/m3]
    D_sf        = 800                           # Seafloor depth [m]

    # Input dimensions
    Lz          = 10000                         # Domain's Z-length
    aspectX     = 3                             # Aspect ratio X
    aspectY     = 3                             # Aspect ratio Y

    # Scales
    rho_sc      = rho_Shale                     # Density scale
    eta_sc      = A_Shale                       # Rock viscosity scale
    L_sc        = ((eta_sc^2)/g/(rho_sc^2))^(1/3)  # Length scale
    PI2         = g*(L_sc^3)*(rho_sc^2)/(eta_sc^2) # Make PI2 = 1 to stabilize
    Lz          = Lz/L_sc                       # Non-dimensionalized Z-length
    v_sc        = rho_sc*g*(L_sc^2)/eta_sc      # Velocity scale
    p_sc        = (eta_sc^2)/(rho_sc*L_sc^2)    # Pressure scale
    p_water     = rho_Water*g*D_sf/p_sc         # Seawater pressure at the top boundary
    p_NMHP      = p_grad_h*Lz*L_sc/p_sc         # Nondimensionalized minimum horizontal pressure
    p_NVP       = p_grad_v*Lz*L_sc/p_sc         # Nondimensionalized vertical pressure

    # Numerics
    nx          = 10                            # Number of blocks in X-direction
    ny          = 10                            # Number of blocks in Y-direction
    nz          = 10                            # Number of blocks in Z-direction
    epsi        = 1e-8                          # Tolerance
    niter       = 5e7                           # Maximum iterations
    nout_iter   = 1000                          # Output iterations
    CFL         = 0.8/sqrt(3)                   # Courant-Friedrichs-Lewy
    Re          = 3/2*sqrt(10)*pi               # Reynolds number
    r           = 1.0                           # Bulk to shear modulus ratio
    err         = 2*epsi                        # Initialize errors
    err_evo1    = []
    err_evo2    = []
    iter        = 0                             # Initialize iteration

    # Pre-processing
    Lx          = aspectX * Lz                  # X-length
    Ly          = aspectY * Lz                  # Y-length
    dx          = Lx/(nx-1)                     # Block X-size
    dy          = Ly/(ny-1)                     # Block Y-size
    dz          = Lz/(nz-1)                     # Block Z-size
    xn          = -dx/2:dx:Lx+dx/2              # Grid X-location
    yn          = -dy/2:dy:Ly+dy/2              # Grid Y-location
    zn          = -dz/2:dz:Lz+dz/2              # Grid Z-location
    xc          = 0:dx:Lx                       # Block center X-location
    yc          = 0:dy:Ly                       # Block center Y-location
    zc          = 0:dz:Lz                       # Block center Z-location
    Xc          = [x for x in xc, y in yc, z in zc] # Mesh X-location
    Yc          = [y for x in xc, y in yc, z in zc] # Mesh Y-location
    Zc          = [z for x in xc, y in yc, z in zc] # Mesh Z-location

    max_lxyz    = max(Lx,Ly,Lz)                 # Maximum block length
    Vpdτ        = min(dx,dy,dz) * CFL           # P-wave velocity * pseudo time step
    Vx          = @zeros(nx+1,ny+1,nz+1)        # Initialize Velocity-X
    Vy          = @zeros(nx+1,ny+1,nz+1)        # Initialize Velocity-Y
    Vz          = @zeros(nx+1,ny+1,nz+1)        # Initialize Velocity-Z
    P           = @zeros(nx,ny,nz)              # Initialize Total Pressure
    τxx         = @ones(nx,ny,nz)               # Initialize Deviatoric Stress XX
    τyy         = @ones(nx,ny,nz)               # Initialize Deviatoric Stress YY
    τzz         = @ones(nx,ny,nz)               # Initialize Deviatoric Stress ZZ
    τxy         = @zeros(nx,ny,nz)              # Initialize Deviatoric Stress XY
    τxz         = @zeros(nx,ny,nz)              # Initialize Deviatoric Stress XZ
    τyz         = @zeros(nx,ny,nz)              # Initialize Deviatoric Stress YZ
    τxx_o       = @ones(nx,ny,nz)               # Initialize Old Deviatoric Stress XX
    τyy_o       = @ones(nx,ny,nz)               # Initialize Old Deviatoric Stress YY
    τzz_o       = @ones(nx,ny,nz)               # Initialize Old Deviatoric Stress ZZ
    τxy_o       = @zeros(nx,ny,nz)              # Initialize Old Deviatoric Stress XY
    τxz_o       = @zeros(nx,ny,nz)              # Initialize Old Deviatoric Stress XZ
    τyz_o       = @zeros(nx,ny,nz)              # Initialize Old Deviatoric Stress YZ
    sigmaxx     = @zeros(nx,ny,nz)              # Initialize Total/Effective Stress XX (total = effective, not consider pore pressure)
    sigmayy     = @zeros(nx,ny,nz)              # Initialize Total/Effective Stress YY
    sigmazz     = @zeros(nx,ny,nz)              # Initialize Total/Effective Stress ZZ
    dVxdx       = @zeros(nx,ny+1,nz+1)          # Initialize dVx/dx
    dVydy       = @zeros(nx+1,ny,nz+1)          # Initialize dVy/dy
    dVzdz       = @zeros(nx+1,ny+1,nz)          # Initialize dVz/dz
    dVxdy       = @zeros(nx+1,ny,nz+1)          # Initialize dVx/dy
    dVxdz       = @zeros(nx+1,ny+1,nz)          # Initialize dVx/dz
    dVydx       = @zeros(nx,ny+1,nz+1)          # Initialize dVy/dx
    dVydz       = @zeros(nx+1,ny+1,nz)          # Initialize dVy/dz
    dVzdx       = @zeros(nx,ny+1,nz+1)          # Initialize dVz/dx
    dVzdy       = @zeros(nx+1,ny,nz+1)          # Initialize dVz/dy
    divV        = @zeros(nx,ny,nz)              # Initialize Divergence of Velocity
    Exx         = @zeros(nx,ny,nz)              # Initialize Strain XX
    Eyy         = @zeros(nx,ny,nz)              # Initialize Strain YY
    Ezz         = @zeros(nx,ny,nz)              # Initialize Strain ZZ
    Exy         = @zeros(nx,ny,nz)              # Initialize Strain XY
    Exz         = @zeros(nx,ny,nz)              # Initialize Strain XZ
    Eyz         = @zeros(nx,ny,nz)              # Initialize Strain YZ
    dPdx        = @zeros(nx-1, ny  , nz)        # Initialize dP/dx
    dPdy        = @zeros(nx  , ny-1, nz)        # Initialize dP/dy
    dPdz        = @zeros(nx  , ny  , nz-1)      # Initialize dP/dz
    dτxxdx      = @zeros(nx-1, ny  , nz)        # Initialize dτxx/dx
    dτyydy      = @zeros(nx  , ny-1, nz)        # Initialize dτyy/dy
    dτzzdz      = @zeros(nx  , ny  , nz-1)      # Initialize dτzz/dz
    dτxydx      = @zeros(nx-1, ny  , nz)        # Initialize dτxy/dx
    dτxydy      = @zeros(nx  , ny-1, nz)        # Initialize dτxy/dy
    dτxzdx      = @zeros(nx-1, ny  , nz)        # Initialize dτxz/dx
    dτxzdz      = @zeros(nx  , ny  , nz-1)      # Initialize dτxz/dz
    dτyzdy      = @zeros(nx  , ny-1, nz)        # Initialize dτyz/dy
    dτyzdz      = @zeros(nx  , ny  , nz-1)      # Initialize dτyz/dz
    Rx          = @zeros(nx-1, ny-1, nz-1)      # Initialize Residual in the X-direction
    Ry          = @zeros(nx-1, ny-1, nz-1)      # Initialize Residual in the Y-direction
    Rz          = @zeros(nx-1, ny-1, nz-1)      # Initialize Residual in the Z-direction
    dVx         = @zeros(nx-1, ny-1 ,nz-1)      # Initialize Velocity-X Difference during convergence
    dVy         = @zeros(nx-1, ny-1, nz-1)      # Initialize Velocity-Y Difference during convergence
    dVz         = @zeros(nx-1, ny-1, nz-1)      # Initialize Velocity-Z Difference during convergence
    P_x         = @zeros(nx  , ny-1, nz-1)      # Initialize Pressure matrix after averaging adjacent row values
    P_y         = @zeros(nx-1, ny  , nz-1)      # Initialize Pressure matrix after averaging adjacent column values
    P_z         = @zeros(nx-1, ny-1, nz  )      # Initialize Pressure matrix after averaging adjacent depth values
    etan        = @ones (nx  , ny  ,nz)         # Initialize Rock Viscosity (Shale)
    etan_x      = @zeros(nx  , ny-1, nz-1)      # Initialize Viscosity matrix after averaging adjacent row values
    etan_y      = @zeros(nx-1, ny, nz-1)        # Initialize Viscosity matrix after averaging adjacent column values
    etan_z      = @zeros(nx-1, ny-1, nz)        # Initialize Viscosity matrix after averaging adjacent depth values
    SaltFraction= @zeros(nx, ny, nz)            # Initialize Salt Fraction
    P_BC        = repeat(reshape(range(1, stop=0, length=nz+1), 1, 1, :), nx+1, ny+1, 1) # Initialize Dimensionless Length, P_BC = grid depth / domain depth, P_BC at seafloor = 0

    # Salt location and density
    radc = ((Xc .- Lx*.5)./4).^2 + ((Yc .- Ly*.5)./4).^2 + ((Zc .- Lz*.5)./1).^2 # Define Salt Location
    SaltFraction[radc .< 0.2/L_sc] .= 1         # Salt Fraction = 1 at Salt Location
    etan[radc .< 0.2/L_sc]         .= A_Salt/A_Shale # Replace Shale Viscosity by Salt Viscosity at Salt Location
    rho_Rock = rho_Shale * ones(nx, ny, nz)     # Initialize Shale Density
    rho_Rock[SaltFraction .== 1.0] .= rho_Salt  # Initialize Salt Density

    # Smear out the coefficients of Stokes equation
    for ism in 1:10
        etan = smooth3D(etan, 1.0)              # Smoothed rock viscosity
    end

    # Calculate the maximum rock viscosity among neighbors
    etanmax = compute_maxloc3D(etan)
    etanmax[1,   :, :  ] = etanmax[2,     :, :    ]
    etanmax[end, :, :  ] = etanmax[end-1, :, :    ]
    etanmax[:,   1, :  ] = etanmax[:,     2, :    ]
    etanmax[:, end, :  ] = etanmax[:, end-1, :    ]
    etanmax[:,   :, 1  ] = etanmax[:,     :, 2    ]
    etanmax[:,   :, end] = etanmax[:,     :, end-1]

    # Pre-processing of numerics
    dτ_Rho = zeros(nx, ny, nz)                  # Pseudo time step / rho
    dτ_Rho = Vpdτ * max_lxyz / Re ./ etanmax
    Gdτ    = zeros(nx, ny, nz)                  # G * pseudo time step
    Gdτ    = (Vpdτ^2) ./ dτ_Rho / (r+2)

    # Pseudo Transient Iterations
    while err > epsi && iter <= niter
        # Compute Velocity Gradients
        @parallel compute_Vgrad!(dVxdx, dVydy, dVzdz, dVxdy, dVxdz, dVydx, dVydz, dVzdx, dVzdy, Vx, Vy, Vz, dx, dy, dz)
        
        # Compute Pressure
        @parallel compute_P!(divV, P, dVxdx, dVydy, dVzdz, Gdτ, r, dx, dy, dz)
        
        # Compute Strain
        @parallel compute_E!(Exx, Eyy, Ezz, Exy, Exz, Eyz, dVxdx, dVydy, dVzdz, dVxdy, dVxdz, dVydx, dVydz, dVzdx, dVzdy)

        # Compute Stress
        @parallel compute_τ!(sigmaxx, sigmayy, sigmazz, P, τxx, τyy, τzz, τxy, τxz, τyz, τxx_o, τyy_o, τzz_o, τxy_o, τxz_o, τyz_o, Exx, Eyy, Ezz, Exy, Exz, Eyz, etan, Gdτ)
        
        # Compute Gradients of Pressure and Deviatoric Stress 
        @parallel compute_Pτgrad!(dPdx, dPdy, dPdz, dτxxdx, dτyydy, dτzzdz, dτxydx, dτxydy, dτxzdx, dτxzdz, dτyzdy, dτyzdz, P, τxx, τyy, τzz, τxy, τxz, τyz, dx, dy, dz)
        
        # Compute Residuals of Stokes Equation
        @parallel compute_dV!(Rx, Ry, Rz, dVx, dVy, dVz, dPdx, dPdy, dPdz, dτxxdx, dτyydy, dτzzdz, dτxydx, dτxydy, dτxzdx, dτxzdz, dτyzdy, dτyzdz, dτ_Rho, PI2, rho_Rock, rho_sc)

        # Compute Velocity
        @parallel compute_V!(Vx, Vy, Vz, dVx, dVy, dVz)
    
        # Apply Boundary Conditions
        @parallel boundary_value!(P_x, P_y, P_z, etan_x, etan_y, etan_z, P, etan)
        @parallel (1:size(P_z,1), 1:size(P_z,2)) TopBottom!(Vz, dz, p_water, p_NVP, P_z, etan_z)
        @parallel (1:size(P_x,2), 1:size(P_x,3)) LeftRight!(Vx, dx, p_NMHP, P_x, etan_x, P_BC)
        @parallel (1:size(P_y,1), 1:size(P_y,3)) FrontBack!(Vy, dy, p_NMHP, P_y, etan_y, P_BC)

        # Update iteration
        iter = iter + 1
        if iter % nout_iter == 0
            Vmin    = minimum(Vx)
            Vmax    = maximum(Vx)
            Pmin    = minimum(P)
            Pmax    = maximum(P)
            norm_Rx = sqrt(sum(Rx.^2)  ) / (Pmax - Pmin) * Lx / sqrt(length(Rx)  )
            norm_Ry = sqrt(sum(Ry.^2)  ) / (Pmax - Pmin) * Ly / sqrt(length(Ry)  )
            norm_Rz = sqrt(sum(Rz.^2)  ) / (Pmax - Pmin) * Lx / sqrt(length(Rz)  )
            norm_dV = sqrt(sum(divV.^2)) / (Vmax - Vmin) * Lx / sqrt(length(divV))
            err     = maximum([norm_Rx, norm_Ry, norm_Rz, norm_dV])
            append!(err_evo1, err)
            append!(err_evo2, iter)
            println("Total steps = $iter, err = $err [norm_Rx=$norm_Rx, norm_Ry=$norm_Ry, norm_Rz=$norm_Rz, norm_dV=$norm_dV]")
        end
        
    end
end

## RUN CODE
SaltInclusion3D = begin SaltInclusion3D();
return;
end